# spark wordcount 1

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!pip install pyspark

In [4]:
# import libraries
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.window import Window
from pyspark.sql.functions import udf, isnan, min, max, sum, count, desc, expr, avg
from pyspark.sql.types import IntegerType, LongType

from pyspark.ml.feature import StandardScaler, VectorAssembler, MinMaxScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier, LogisticRegressionModel, GBTClassificationModel, RandomForestClassificationModel

import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
import pandas as pd

In [5]:
from pyspark import SparkContext
# creating spark context
sc = SparkContext('local', 'WordCount App')

In [ ]:
# loading text from file
with open('idiot.txt') as src:
    text = src.readlines()

# making RDD from text lines
text_rdd = sc.parallelize(text)

# counting word entries and storing result as RDD
wc_rdd = text_rdd.flatMap(lambda line: line.split()) \
                    .map(lambda word: (word, 1)) \
                    .reduceByKey(lambda x, y: x + y)

# output 10 most frequent words
most_frequent = wc_rdd.map(lambda e: e[::-1]).top(10)
print(most_frequent)

# # getting result back to client
wc = wc_rdd.collect()
#
 # output 10 most frequent words
print(sorted(wc, key=lambda e: -e[-1])[:10])

# sparksql

In [ ]:
from google.colab import files

uploaded = files.upload()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master('local') \
            .appName('SQL App').getOrCreate()
sc = spark.sparkContext

In [ ]:
!cat sales_header.csv

In [12]:
def parse_row(line):
    # TODO: parse row values
    return line.split(',')

sales_rdd = sc.textFile('sales.csv').map(parse_row)

In [ ]:
with open('sales_header.csv') as src:
    sales_header = src.read().strip().split(',')

sales_df = sales_rdd.toDF(sales_header)
sales_df.registerTempTable('sales')

sales_total_df = spark.sql("SELECT COUNT(1) FROM sales")
print(sales_total_df.rdd.collect())

# TODO: print revenue by Country and State